In [ ]:
import pandas as pd
import numpy as np
from pyspark.sql.functions import when, lit, to_date
import plotly.graph_objects as go

import sys
sys.path.append('../')
import pre_process
import constants
import utils


# Motor EDA

In [ ]:
period_start = '2017-03-01'
period_end = '2017-09-01'

notification_df = pre_process.create_notifications_df(local_path='../')
del notification_df['endTime']

# only want dates between '2017-03-01' and '2017-09-01'
filtered_notification_df = notification_df[(notification_df['startTime'] >= pd.to_datetime(period_start)) & (notification_df['startTime'] <= pd.to_datetime(period_end))]
filtered_notification_df

In [ ]:
motor_subsets = pre_process.run(mode=constants.MOTOR, local_path='../')
print(motor_subsets.keys())

# DE Bearing Temp

In [ ]:
de_bearing_temp = pre_process.filter_and_resample(motor_subsets['de_bearing_temp'])

In [ ]:
utils.plot_with_not(de_bearing_temp, filtered_notification_df)

# Winding Temperature

In [ ]:
winding_temperature = pre_process.filter_and_resample(motor_subsets['winding_temperature'])

In [ ]:
utils.plot_with_not(winding_temperature, filtered_notification_df)

# NDE Bearing Temp

In [ ]:
nde_bearing_temp = pre_process.filter_and_resample(motor_subsets['nde_bearing_temp'])

In [ ]:
utils.plot_with_not(nde_bearing_temp, filtered_notification_df)

# NDE Vibration

In [ ]:
nde_vibration = pre_process.filter_and_resample(motor_subsets['nde_vibration'])

In [ ]:
utils.plot_with_not(nde_vibration, filtered_notification_df)

# RPM

In [ ]:
rpm = pre_process.filter_and_resample(motor_subsets['rpm'])

In [ ]:
utils.plot_with_not(rpm, filtered_notification_df)

In [ ]:
from functools import reduce

def clean_motor_off(df_sensor_list):
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['timestamp'], how='outer'), df_sensor_list)

    for col in df_merged.columns:
        df_merged[col] = df_merged.apply(lambda row: np.NaN if row['rpm'] == 0 else row[col], axis=1).interpolate() # method='polynomial', order=2
        
    return df_merged

In [ ]:
df = clean_motor_off([de_bearing_temp, nde_bearing_temp, nde_vibration, winding_temperature, rpm])
df.head()

In [ ]:
utils.plot_with_not(df, filtered_notification_df)

In [ ]:
df.tail()

In [ ]:
compact_df = pd.DataFrame()

'''def merge_sensors(column_name, row):
    compact_df[column_name] = df.apply(lambda row : (row[column_name+' A']+row['DE Bearing Temp B'])/2, axis=1)'''

compact_df['DE Bearing Temp'] = df.apply(lambda row : (row['DE Bearing Temp A']+row['DE Bearing Temp B'])/2, axis=1)
compact_df['NDE Bearing Temp'] = df.apply(lambda row : (row['NDE Bearing Temp A']+row['NDE Bearing Temp B'])/2, axis=1)
compact_df['NDE Vibration'] = df.apply(lambda row : (row['NDE Vibration X plane']+row['NDE Vibration Y plane']+row['NDE Vibration X plane '])/3, axis=1)
compact_df['Winding Temperature'] = df.apply(lambda row : (row['Winding "U" Temperature']+row['Winding "V" Temperature']+row['Winding "W" Temperature'])/3, axis=1)
compact_df['rpm'] = df['rpm']



compact_df.tail()

In [ ]:
import plotly.express as px

fig = px.imshow(compact_df.corr())
fig.show()